## Getting the dataset of 15 family of mushroom

In [1]:
from PIL import Image
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset
import os

In [2]:
path = "C:/Users/mbrei/OneDrive/Bureau/ChampIA/Data/data_image"

We create a class to get the path of each image according to family and data type to transform it in tensors


In [3]:
class MushroomDataset(Dataset):
        def __init__(self, root_dir, data_type, transform=None):
                """
                Choisir un répertoire racine et une transformation à appliquer aux images
                """
                self.root_dir = root_dir
                self.data_type = data_type 
                self.transform = transform
                self.data = []
                self.classes = {}
                self._load_images()
                
        def _load_images(self):
                """
                Récupérer les chemins de chaque image via différents dossiers avec l'étiquette de la famille de champignons
                """
                class_idx = 0
                for family_name in os.listdir(self.root_dir):
                        family_dir = os.path.join(self.root_dir, family_name)
                        if os.path.isdir(family_dir):
                                data_type_dir = os.path.join(family_dir, 'data', self.data_type, family_name).replace('\\', '/')
                                data_type_dir_champ = os.path.join(family_dir, 'data', self.data_type, f"mushrooms {family_name}").replace('\\', '/')
                                if os.path.exists(data_type_dir):
                                        target_dir = data_type_dir
                                elif os.path.exists(data_type_dir_champ):
                                        target_dir = data_type_dir_champ
                                else:
                                        continue
                                if family_name not in self.classes:
                                        self.classes[family_name] = class_idx
                                        class_idx += 1
                                for img_file in os.listdir(target_dir):
                                        img_path = os.path.join(target_dir, img_file).replace('\\', '/')
                                        if img_file.lower().endswith(('.jpg')):
                                                self.data.append((img_path, family_name))
        
        def __len__(self):
                return len(self.data)
        
        def __getitem__(self, idx):
                img_path, family_name = self.data[idx]
                image = Image.open(img_path).convert("RGB")
                
                if self.transform:
                        image = self.transform(image)
                
                # Convertir le nom de famille en index numérique
                label = self.classes[family_name]
                
                return image, label

In [4]:
# Data with only train
transform = transforms.Compose([transforms.ToTensor()])

train_data = MushroomDataset(root_dir=path, data_type='train', transform=transform)
test_data = MushroomDataset(root_dir=path, data_type='test', transform=transform)
val_data = MushroomDataset(root_dir=path, data_type='val', transform=transform)

In [5]:
train_data[0]

(tensor([[[0.8471, 0.7647, 0.7608,  ..., 0.4745, 0.3922, 0.3176],
          [0.7843, 0.7098, 0.7765,  ..., 0.3529, 0.3333, 0.2353],
          [0.7765, 0.7294, 0.6471,  ..., 0.4824, 0.4275, 0.3765],
          ...,
          [0.5294, 0.5647, 0.3843,  ..., 0.5294, 0.3686, 0.2275],
          [0.3373, 0.3216, 0.3804,  ..., 0.1725, 0.1451, 0.4275],
          [0.3020, 0.2157, 0.1216,  ..., 0.3294, 0.4706, 0.5059]],
 
         [[0.8353, 0.7176, 0.6706,  ..., 0.4078, 0.3686, 0.3216],
          [0.7569, 0.6549, 0.6784,  ..., 0.2588, 0.2667, 0.1843],
          [0.7294, 0.6549, 0.5451,  ..., 0.3569, 0.3059, 0.2510],
          ...,
          [0.3373, 0.3765, 0.1882,  ..., 0.4784, 0.2667, 0.0706],
          [0.1725, 0.1569, 0.2196,  ..., 0.1216, 0.0667, 0.3216],
          [0.1569, 0.0784, 0.0078,  ..., 0.2549, 0.4196, 0.4824]],
 
         [[0.6745, 0.6314, 0.6392,  ..., 0.3294, 0.3137, 0.2902],
          [0.6471, 0.6039, 0.6627,  ..., 0.2510, 0.2902, 0.2235],
          [0.6824, 0.6392, 0.5490,  ...,

In [6]:
image, label = train_data.__getitem__(1200)
image.shape, label

(torch.Size([3, 128, 128]), 2)

In [22]:
# Transformation des images pour les amener à la bonne forme et les normaliser
transform = transforms.Compose([
  transforms.Resize((128, 128)),  # Assurer que l'image est bien de taille 128x128\n",
  transforms.ToTensor(),  # Convertir l'image en un tenseur\n",
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalisation\n",
  ]),

In [23]:
BATCH_SIZE = 16

# Supposons que train_data est un objet Dataset personnalisé, donc on l'enveloppe dans un DataLoader
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

In [24]:
import torch.nn as nn
import torch.optim as optim

class SimpleCNN(nn.Module):

        def __init__(self, num_classes):
            super(SimpleCNN, self).__init__()
            
            # Définir les couches du CNN
            self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
            self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
            self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
            
            # MaxPooling pour réduire la taille de l'image
            self.pool = nn.MaxPool2d(2, 2)
            
            # Couches Fully Connected
            self.fc1 = nn.Linear(64 * 16 * 16, 512)  # Assurez-vous que les dimensions correspondent
            self.fc2 = nn.Linear(512, num_classes)  # Nombre de classes
            
        def forward(self, x):
            # Appliquer les convolutions et les max pooling
            x = self.pool(torch.relu(self.conv1(x)))
            x = self.pool(torch.relu(self.conv2(x)))
            x = self.pool(torch.relu(self.conv3(x)))
            # Applatir l'image pour la passer dans les couches fully connected
            x = x.view(-1, 64 * 16 * 16)
            
            # Appliquer les couches fully connected
            x = torch.relu(self.fc1(x))
            x = self.fc2(x)
            
            return x

In [25]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("runs/experiment3")
# writer.add_text("Learning rate de 0")
 

In [27]:
LR = 0.005
EPOCH = 15
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
num_classes = len(train_data.classes)
model = SimpleCNN(num_classes=num_classes)
hparams = {'lr': LR, 'batch_size': BATCH_SIZE, 'optimizer': 'Adam'}

In [32]:
from torchinfo import summary
summary(model, input_size=(BATCH_SIZE, 3, 128, 128))

Layer (type:depth-idx)                   Output Shape              Param #
SimpleCNN                                [16, 13]                  --
├─Conv2d: 1-1                            [16, 16, 128, 128]        448
├─MaxPool2d: 1-2                         [16, 16, 64, 64]          --
├─Conv2d: 1-3                            [16, 32, 64, 64]          4,640
├─MaxPool2d: 1-4                         [16, 32, 32, 32]          --
├─Conv2d: 1-5                            [16, 64, 32, 32]          18,496
├─MaxPool2d: 1-6                         [16, 64, 16, 16]          --
├─Linear: 1-7                            [16, 512]                 8,389,120
├─Linear: 1-8                            [16, 13]                  6,669
Total params: 8,419,373
Trainable params: 8,419,373
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 858.90
Input size (MB): 3.15
Forward/backward pass size (MB): 58.79
Params size (MB): 33.68
Estimated Total Size (MB): 95.61

In [ ]:
#!python -m tensorboard.main --logdir=Code/runs

^C


In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# Définir l'appareil (GPU ou CPU)
model.to(DEVICE)

# Initialisation des meilleures accuracies
best_train_acc = 0.0
best_val_acc = 0.0

# Entraîner le modèle avec validation
for epoch in range(EPOCH):
    # Phase d'entraînement
    model.train()  
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        running_loss += loss.item()
        
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total

    # Phase de validation
    model.eval()  
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():  
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            val_loss += loss.item()

    val_epoch_loss = val_loss / len(val_loader)
    val_epoch_acc = 100 * val_correct / val_total

    # Mettre à jour les meilleures accuracies
    if epoch_acc > best_train_acc:
        best_train_acc = epoch_acc

    if val_epoch_acc > best_val_acc:
        best_val_acc = val_epoch_acc

    # Affichage des métriques
    print(f"Epoch {epoch+1}/{EPOCH} - Loss: {epoch_loss:.4f} - Acc: {epoch_acc:.4f} | "
          f"Val Loss: {val_epoch_loss:.4f} - Val Acc: {val_epoch_acc:.4f} | ")
    
    # Enregistrement dans TensorBoard
    writer.add_scalar("Loss/train", epoch_loss, epoch)
    writer.add_scalar("Accuracy/train", epoch_acc, epoch)
    writer.add_scalar("Loss/val", val_epoch_loss, epoch)
    writer.add_scalar("Accuracy/val", val_epoch_acc, epoch)


description = f"""
# Expérience d'entraînement

## Paramètres :
- Learning Rate : {LR}
- Batch Size : {BATCH_SIZE}
- Optimiseur : {optimizer}

## Meilleures Performances :
- Best Train Accuracy : {best_train_acc:.4f}%
- Best Validation Accuracy : {best_val_acc:.4f}%

## Architecture du modèle :
{summary(model, input_size=(BATCH_SIZE, 3, 128, 128))}
"""

writer.add_text("Documentation du modèle :", description)
    
writer.close()


Epoch 1/15 - Loss: 2.4628 - Acc: 11.7197 | Val Loss: 2.4536 - Val Acc: 12.8306 | 
Epoch 2/15 - Loss: 2.4436 - Acc: 11.7056 | Val Loss: 2.4530 - Val Acc: 12.8306 | 
Epoch 3/15 - Loss: 2.4441 - Acc: 12.1302 | Val Loss: 2.4511 - Val Acc: 12.8306 | 
Epoch 4/15 - Loss: 2.4415 - Acc: 11.9887 | Val Loss: 2.4546 - Val Acc: 12.8306 | 
Epoch 5/15 - Loss: 2.4407 - Acc: 12.4841 | Val Loss: 2.4573 - Val Acc: 11.1986 | 
Epoch 6/15 - Loss: 2.4405 - Acc: 12.1019 | Val Loss: 2.4520 - Val Acc: 12.8306 | 
Epoch 7/15 - Loss: 2.4416 - Acc: 12.0736 | Val Loss: 2.4540 - Val Acc: 12.8306 | 
Epoch 8/15 - Loss: 2.4411 - Acc: 12.5548 | Val Loss: 2.4515 - Val Acc: 12.8306 | 
Epoch 9/15 - Loss: 2.4395 - Acc: 12.2435 | Val Loss: 2.4538 - Val Acc: 12.8306 | 
Epoch 10/15 - Loss: 2.4397 - Acc: 12.3992 | Val Loss: 2.4499 - Val Acc: 12.8306 | 
Epoch 11/15 - Loss: 2.4402 - Acc: 12.5548 | Val Loss: 2.4528 - Val Acc: 12.8306 | 
Epoch 12/15 - Loss: 2.4394 - Acc: 12.5548 | Val Loss: 2.4536 - Val Acc: 12.8306 | 
Epoch 13/15 -

In [17]:
torch.save(model.state_dict(), "./Model/mushroom_cnn_001.pth")

In [31]:
def test_accuracy(model, test_loader, device):
    model.eval()  # Mettre le modèle en mode évaluation
    correct = 0
    total = 0

    with torch.no_grad():  # Désactiver la gradation
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test data: {accuracy:.2f}%')

test_accuracy(model, test_loader, DEVICE)

Accuracy of the model on the test data: 42.86%
